# Kaggle Competion - Titanic: Machine Learning from Disaster

## Run Configurations

In [38]:
MODEL_CODE = "GNB"                   # DTC or GNB or SVM
N_SPLIT = 10                         # k-Fold k value
REPORT_NAME = "REP_" + MODEL_CODE    # Reporting file name

## Preprocessing

In [39]:
import pandas as pd
import numpy as np

In [40]:
df = pd.read_csv("data/train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [41]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [42]:
for col in df.columns:
    print(f"{col} - {df[df[col].isna()].shape}")

PassengerId - (0, 12)
Survived - (0, 12)
Pclass - (0, 12)
Name - (0, 12)
Sex - (0, 12)
Age - (177, 12)
SibSp - (0, 12)
Parch - (0, 12)
Ticket - (0, 12)
Fare - (0, 12)
Cabin - (687, 12)
Embarked - (2, 12)


In [43]:
c1 = df.drop(["Name", "Cabin", "Ticket"], axis=1)

In [44]:
c2 = c1.dropna()

In [45]:
for col in c2.columns:
    print(f"{col} - {c2[c2[col].isna()].shape}")

PassengerId - (0, 9)
Survived - (0, 9)
Pclass - (0, 9)
Sex - (0, 9)
Age - (0, 9)
SibSp - (0, 9)
Parch - (0, 9)
Fare - (0, 9)
Embarked - (0, 9)


In [46]:
c2

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
885,886,0,3,female,39.0,0,5,29.1250,Q
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [47]:
c3 = c2.drop("Survived", axis=1)
c3

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
885,886,3,female,39.0,0,5,29.1250,Q
886,887,2,male,27.0,0,0,13.0000,S
887,888,1,female,19.0,0,0,30.0000,S
889,890,1,male,26.0,0,0,30.0000,C


In [48]:
y = c2["Survived"]
y

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 712, dtype: int64

In [49]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [50]:
# 2 and 7 columns are the Age and Embarked columns of c3
ctf = ColumnTransformer([("train-data-encoder", OneHotEncoder(), [2,7])], remainder="passthrough")

In [51]:
c4a = np.array(ctf.fit_transform(c3), dtype=np.str)
c4 = pd.DataFrame(c4a)
c4

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,22.0,1.0,0.0,7.25
1,1.0,0.0,1.0,0.0,0.0,2.0,1.0,38.0,1.0,0.0,71.2833
2,1.0,0.0,0.0,0.0,1.0,3.0,3.0,26.0,0.0,0.0,7.925
3,1.0,0.0,0.0,0.0,1.0,4.0,1.0,35.0,1.0,0.0,53.1
4,0.0,1.0,0.0,0.0,1.0,5.0,3.0,35.0,0.0,0.0,8.05
...,...,...,...,...,...,...,...,...,...,...,...
707,1.0,0.0,0.0,1.0,0.0,886.0,3.0,39.0,0.0,5.0,29.125
708,0.0,1.0,0.0,0.0,1.0,887.0,2.0,27.0,0.0,0.0,13.0
709,1.0,0.0,0.0,0.0,1.0,888.0,1.0,19.0,0.0,0.0,30.0
710,0.0,1.0,1.0,0.0,0.0,890.0,1.0,26.0,0.0,0.0,30.0


In [52]:
X = c4

# for k-Fold cross validation convenience
Xv = X.values
yv = y.values

## Model Evaluation with k-Fold Cross Validation

In [53]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

def create_model(code):
    if code == "DTC":
        return DecisionTreeClassifier()
    elif code == "GNB":
        return GaussianNB()
    elif code == "SVM":
        return SVC(kernel='linear')
    return None

In [54]:
from sklearn.model_selection import KFold         # 10-Fold Cross Validation
from sklearn.preprocessing import StandardScaler  # Standardizing input data
from sklearn.metrics import confusion_matrix      # to calculate confusion matrix
from sklearn.metrics import accuracy_score        # to calculate accuracy
from sklearn.metrics import precision_score       # to calculate precision
from sklearn.metrics import recall_score          # to calculate recall
from sklearn.metrics import f1_score              # to calculate f1 score
import time                                       # to measure elapsed time
import os                                         # to create a folder for reporting

In [55]:
if not os.path.exists('reports'):
    os.makedirs('reports')

file = open(f"reports/{REPORT_NAME}_EVAL.txt", "w")

start = time.process_time()

# initializing variables for kFold run and F1_Score
current_fold = 0
total_acc = 0
total_prc = 0
total_rec = 0
total_f1s = 0

for train_index, test_index in KFold(N_SPLIT).split(Xv):
    current_fold += 1
    
    x_train, x_test = Xv[train_index], Xv[test_index]
    y_train, y_test = yv[train_index], yv[test_index]
    
    if MODEL_CODE == "SVM":
        sc = StandardScaler()
        x_train = sc.fit_transform(x_train)  # Fit and then transform. Process to fit is learnt here.
        x_test = sc.transform(x_test)
    
    model=create_model(MODEL_CODE)  # model will be discarded after the fold is finished
    
    model.fit(x_train, y_train)
            
    y_pred = model.predict(x_test)
    y_pred = (y_pred > 0.5)
    
    cfm = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    prc = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    
    f1s = f1_score(y_test, y_pred, average="macro")
    # average='macro' : Calculate metrics for each label, and find their unweighted mean. 
    # This does not take label imbalance into account
    
    ITR_EVAL_STR  = f"{str(current_fold).rjust(2)}-FOLD\t| Precision\t= {prc}\n"
    ITR_EVAL_STR += f"\t\t  Recall\t= {rec}\n"
    ITR_EVAL_STR += f"\t\t  F1 Score\t= {f1s}\n"
    
    print(ITR_EVAL_STR)
    file.write(ITR_EVAL_STR + "\n")
    
    total_acc += acc
    total_prc += prc
    total_rec += rec
    total_f1s += f1s


end = time.process_time()

# AVG_ACCURACY_STR = f"AVG_ACCURACY\t= {total_acc/N_SPLIT}"
AVG_PRECISION_STR = f"AVG_PRECISION\t= {total_prc/N_SPLIT}"
AVG_RECALL_STR = f"AVG_RECALL\t\t= {total_rec/N_SPLIT}"
AVG_F1_SCORE_STR = f"AVG_F1_SCORE\t= {total_f1s/N_SPLIT}\n"
EXEC_TIME_STR = f"EXEC_TIME\t= {end-start}s"

# print(AVG_ACCURACY_STR)
print(AVG_PRECISION_STR)
print(AVG_RECALL_STR)
print(AVG_F1_SCORE_STR)
print(EXEC_TIME_STR)

# file.write(AVG_ACCURACY_STR + "\n")
file.write(AVG_PRECISION_STR + "\n")
file.write(AVG_RECALL_STR + "\n")
file.write(AVG_F1_SCORE_STR + "\n")
file.write(EXEC_TIME_STR + "\n")
    
file.close()

 1-FOLD	| Precision	= 0.6428571428571429
		  Recall	= 0.6428571428571429
		  F1 Score	= 0.7077922077922079

 2-FOLD	| Precision	= 0.4230769230769231
		  Recall	= 0.6875
		  F1 Score	= 0.6638655462184873

 3-FOLD	| Precision	= 0.7727272727272727
		  Recall	= 0.5666666666666667
		  F1 Score	= 0.726923076923077

 4-FOLD	| Precision	= 0.7714285714285715
		  Recall	= 0.75
		  F1 Score	= 0.7605633802816902

 5-FOLD	| Precision	= 0.6666666666666666
		  Recall	= 0.8
		  F1 Score	= 0.7452153110047847

 6-FOLD	| Precision	= 0.7575757575757576
		  Recall	= 0.7575757575757576
		  F1 Score	= 0.773524720893142

 7-FOLD	| Precision	= 0.8148148148148148
		  Recall	= 0.6666666666666666
		  F1 Score	= 0.7691056910569105

 8-FOLD	| Precision	= 0.7241379310344828
		  Recall	= 0.7777777777777778
		  F1 Score	= 0.7936046511627908

 9-FOLD	| Precision	= 0.8
		  Recall	= 0.6896551724137931
		  F1 Score	= 0.7908249158249159

10-FOLD	| Precision	= 0.7777777777777778
		  Recall	= 0.8076923076923077
		  F1 Score	

## Model Training

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [58]:
start = time.process_time()

In [59]:
if MODEL_CODE == "SVM":
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)  # Fit and then transform. Process to fit is learnt here.
    X_test = sc.transform(X_test)

In [60]:
model = create_model(MODEL_CODE)
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [61]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

In [62]:
end = time.process_time()

In [63]:
acc = accuracy_score(y_test, y_pred)
prc = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1s = f1_score(y_test, y_pred, average="macro")

In [64]:
if not os.path.exists('reports'):
    os.makedirs('reports')

file = open(f"reports/{REPORT_NAME}_TRAIN.txt", "w")

# ACCURACY_STR = f"ACCURACY\t= {acc}"
PRECISION_STR = f"PRECISION\t= {prc}"
RECALL_STR = f"RECALL\t\t= {rec}"
F1_SCORE_STR = f"F1_SCORE\t= {f1s}\n"
EXEC_TIME_STR = f"EXEC_TIME\t= {end-start}s"

# print(ACCURACY_STR)
print(PRECISION_STR)
print(RECALL_STR)
print(F1_SCORE_STR)
print(EXEC_TIME_STR)

# file.write(ACCURACY_STR + "\n")
file.write(PRECISION_STR + "\n")
file.write(RECALL_STR + "\n")
file.write(F1_SCORE_STR + "\n")
file.write(EXEC_TIME_STR + "\n")

file.close()

PRECISION	= 0.7543859649122807
RECALL		= 0.7049180327868853
F1_SCORE	= 0.7691686844229217

EXEC_TIME	= 0.0625s


## Predictions

In [65]:
td = pd.read_csv("data/test.csv")
td.shape

(418, 11)

In [66]:
tc1 = td.drop(["Name", "Cabin", "Ticket"], axis=1)
tc1

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,NaN,0,0,8.0500,S


In [67]:
for col in tc1.columns:
    print(f"{col} - {tc1[tc1[col].isna()].shape}")

PassengerId - (0, 8)
Pclass - (0, 8)
Sex - (0, 8)
Age - (86, 8)
SibSp - (0, 8)
Parch - (0, 8)
Fare - (1, 8)
Embarked - (0, 8)


In [69]:
tc2 = tc1.replace(np.nan, 0)
tc2

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
409,1301,3,female,3.0,1,1,13.7750,S
411,1303,1,female,37.0,1,0,90.0000,Q
412,1304,3,female,28.0,0,0,7.7750,S
414,1306,1,female,39.0,0,0,108.9000,C


In [70]:
# 2 and 7 columns are the Age and Embarked columns of tc2
ctf2 = ColumnTransformer([("test-data-encoder", OneHotEncoder(), [2,7])], remainder="passthrough")

In [71]:
tc3a = np.array(ctf2.fit_transform(tc2), dtype=np.str)
tc3 = pd.DataFrame(tc3a)
tc3

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,1.0,0.0,1.0,0.0,892.0,3.0,34.5,0.0,0.0,7.8292
1,1.0,0.0,0.0,0.0,1.0,893.0,3.0,47.0,1.0,0.0,7.0
2,0.0,1.0,0.0,1.0,0.0,894.0,2.0,62.0,0.0,0.0,9.6875
3,0.0,1.0,0.0,0.0,1.0,895.0,3.0,27.0,0.0,0.0,8.6625
4,1.0,0.0,0.0,0.0,1.0,896.0,3.0,22.0,1.0,1.0,12.2875
...,...,...,...,...,...,...,...,...,...,...,...
326,1.0,0.0,0.0,0.0,1.0,1301.0,3.0,3.0,1.0,1.0,13.775
327,1.0,0.0,0.0,1.0,0.0,1303.0,1.0,37.0,1.0,0.0,90.0
328,1.0,0.0,0.0,0.0,1.0,1304.0,3.0,28.0,0.0,0.0,7.775
329,1.0,0.0,1.0,0.0,0.0,1306.0,1.0,39.0,0.0,0.0,108.9


In [72]:
if MODEL_CODE == "SVM":
    sc = StandardScaler()
    tc3 = sc.fit_transform(tc3)  # Fit and then transform. Process to fit is learnt here.

In [77]:
y_pred = model.predict(tc3)

0       892.0
1       893.0
2       894.0
3       895.0
4       896.0
        ...  
326    1301.0
327    1303.0
328    1304.0
329    1306.0
330    1307.0
Name: 5, Length: 331, dtype: object

In [81]:
predictions = pd.DataFrame(td["PassengerId"], columns=["PassengerId"])
predictions["Survived"] = y_pred
predictions

,PassengerId,Survived
0,NaN,0
1,NaN,1
2,NaN,0
3,NaN,0
4,NaN,1
...,...,...
326,NaN,1
327,NaN,1
328,NaN,1
329,NaN,1


In [37]:
predictions.to_csv(f"pred/predctions_{MODEL_CODE}.csv", index=False)